<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hive-Data-Definition-Language-(DDL)" data-toc-modified-id="Hive-Data-Definition-Language-(DDL)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hive Data Definition Language (DDL)</a></span></li><li><span><a href="#Data-SELECT-and-JOIN" data-toc-modified-id="Data-SELECT-and-JOIN-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data <code>SELECT</code> and <code>JOIN</code></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Nested-SELECT" data-toc-modified-id="Nested-SELECT-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Nested <code>SELECT</code></a></span></li><li><span><a href="#Aggregation-and-sampling" data-toc-modified-id="Aggregation-and-sampling-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Aggregation and sampling</a></span></li></ul></li></ul></li><li><span><a href="#Analytic-functions" data-toc-modified-id="Analytic-functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Analytic functions</a></span></li></ul></div>

In [4]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.
    appName("test").
    enableHiveSupport().
    config("spark.sql.warehouse.dir","/user/hive/warehouse").
    getOrCreate()
)

spark.sparkContext.getConf().getAll()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[('spark.driver.port', '11163'),
 ('spark.app.name', 'test'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '192.168.199.77'),
 ('spark.hadoop.fs.default.name', 'hdfs://localhost:9000'),
 ('spark.app.id', 'local-1638883476000'),
 ('spark.sql.warehouse.dir', '/user/hive/warehouse'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.hadoop.fs.defaultFS', 'hdfs://localhost:9000'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.startTime', '1638883474378'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [8]:
path = "file:///home/sergey/Py_SparkDataFrame_edx_CS105_CS110_CS120/data/201408_status_data.csv"
!hdfs dfs -head {path}

"station_id","bikes_available","docks_available","time"
"2","12","15","2014-03-01 00:00:02"
"2","12","15","2014-03-01 00:01:03"
"2","12","15","2014-03-01 00:02:03"
"2","12","15","2014-03-01 00:03:02"
"2","12","15","2014-03-01 00:04:03"
"2","12","15","2014-03-01 00:05:02"
"2","12","15","2014-03-01 00:06:03"
"2","12","15","2014-03-01 00:07:02"
"2","13","14","2014-03-01 00:08:03"
"2","13","14","2014-03-01 00:09:02"
"2","13","14","2014-03-01 00:10:03"
"2","13","14","2014-03-01 00:11:03"
"2","13","14","2014-03-01 00:12:02"
"2","13","14","2014-03-01 00:13:02"
"2","13","14","2014-03-01 00:14:02"
"2","13","14","2014-03-01 00:15:02"
"2","13","14","2014-03-01 00:16:03"
"2","13","14","2014-03-01 00:17:03"
"2","13","14","2014-03-01 00:18:03"
"2","13","14","2014-03-01 00:19:02"
"2","13","14","2014-03-01 00:20:02"
"2","13","14","2014-03-01 00:21:02"
"2","13","14","2014-03-01 00:22:02"
"2","13","14","2014-03-01 00:23:02"
"2","13","14","2014-03-01 00:24:02"
"2","13","14","2014-03-01 00:25:03"
"2","13"

In [33]:
schema = "station_id int, bikes_available int, docks_available int, time timestamp"
df = spark.read.csv(path, header=True, schema=schema).cache()
df.printSchema()

root
 |-- station_id: integer (nullable = true)
 |-- bikes_available: integer (nullable = true)
 |-- docks_available: integer (nullable = true)
 |-- time: timestamp (nullable = true)



In [12]:
df.count()

18342210

In [11]:
df.show(5)

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|         2|             12|             15|2014-03-01 00:00:02|
|         2|             12|             15|2014-03-01 00:01:03|
|         2|             12|             15|2014-03-01 00:02:03|
|         2|             12|             15|2014-03-01 00:03:02|
|         2|             12|             15|2014-03-01 00:04:03|
+----------+---------------+---------------+-------------------+
only showing top 5 rows



In [13]:
df.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- bikes_available: string (nullable = true)
 |-- docks_available: string (nullable = true)
 |-- time: string (nullable = true)



In [14]:
df.selectExpr(('2*bikes_available')).show(5)

+---------------------+
|(2 * bikes_available)|
+---------------------+
|                 24.0|
|                 24.0|
|                 24.0|
|                 24.0|
|                 24.0|
+---------------------+
only showing top 5 rows



This does not work!!! :

In [14]:
#df.select(('2*bikes_available')).show(5)

In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
bikes_double = udf(lambda x: 2*int(x), IntegerType())
df.select(bikes_double('bikes_available').alias('2 x bikes_available')).show(5)

+-------------------+
|2 x bikes_available|
+-------------------+
|                 24|
|                 24|
|                 24|
|                 24|
|                 24|
+-------------------+
only showing top 5 rows



In [15]:
df.select(2*df.bikes_available).show(5)

+---------------------+
|(bikes_available * 2)|
+---------------------+
|                   24|
|                   24|
|                   24|
|                   24|
|                   24|
+---------------------+
only showing top 5 rows



# Hive Data Definition Language (DDL)

In [35]:
spark.catalog.dropTempView('tb1')
df.createTempView('tb1')
spark.sql("DESCRIBE FORMATTED tb1").show(truncate=False);

+---------------+---------+-------+
|col_name       |data_type|comment|
+---------------+---------+-------+
|station_id     |int      |null   |
|bikes_available|int      |null   |
|docks_available|int      |null   |
|time           |timestamp|null   |
+---------------+---------+-------+



In [19]:
spark.sql('show databases').show()

+---------------+
|      namespace|
+---------------+
|            cbr|
|        default|
|hive_essentials|
|    pluralsight|
|         rostel|
|    rostel_test|
+---------------+



In [21]:
spark.sql('DESCRIBE DATABASE default').show(truncate=False)

+-------------+-----------------------------------------+
|info_name    |info_value                               |
+-------------+-----------------------------------------+
|Database Name|default                                  |
|Comment      |Default Hive database                    |
|Location     |hdfs://localhost:9000/user/hive/warehouse|
|Owner        |public                                   |
+-------------+-----------------------------------------+



In [23]:
spark.sql('USE default');

In [24]:
context.sql('show tables').show()

+---------+-----------+
|tableName|isTemporary|
+---------+-----------+
|      tb1|       true|
+---------+-----------+



In [25]:
df.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- bikes_available: string (nullable = true)
 |-- docks_available: string (nullable = true)
 |-- time: string (nullable = true)



In [26]:
context.sql('desc tb1').show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|     station_id|   string|       |
|bikes_available|   string|       |
|docks_available|   string|       |
|           time|   string|       |
+---------------+---------+-------+



# Data `SELECT` and `JOIN`  

The `SELECT` statement is quite often used with `FROM , DISTINCT , WHERE , and LIMIT`
keywords.

In [26]:
spark.sql('select * from tb1 where bikes_available >= 10').show(5)

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|         2|             12|             15|2014-03-01 00:00:02|
|         2|             12|             15|2014-03-01 00:01:03|
|         2|             12|             15|2014-03-01 00:02:03|
|         2|             12|             15|2014-03-01 00:03:02|
|         2|             12|             15|2014-03-01 00:04:03|
+----------+---------------+---------------+-------------------+
only showing top 5 rows



In [27]:
spark.sql('select count(distinct station_id) number_of_dockstations from tb1').show()

+----------------------+
|number_of_dockstations|
+----------------------+
|                    70|
+----------------------+



In [37]:
spark.sql('select distinct station_id as dist_sid from tb1 order by dist_sid desc limit 5').show()

+--------+
|dist_sid|
+--------+
|      84|
|      83|
|      82|
|      80|
|      77|
+--------+



In [38]:
spark.sql('select max(station_id) from tb1;').show()

+---------------+
|max(station_id)|
+---------------+
|             84|
+---------------+



### Nested `SELECT`

In [39]:
spark.sql(
    '''
    WITH tb2 AS
    (SELECT * FROM tb1 WHERE station_id >=80)
    SELECT * from tb2 LIMIT 5
    '''
).show()

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|        80|              8|              7|2014-03-01 00:00:02|
|        80|              8|              7|2014-03-01 00:01:03|
|        80|              8|              7|2014-03-01 00:02:03|
|        80|              8|              7|2014-03-01 00:03:02|
|        80|              8|              7|2014-03-01 00:04:03|
+----------+---------------+---------------+-------------------+



In [24]:
spark.sql('SELECT * FROM (SELECT * FROM tb1 WHERE station_id >= 80) tb2').show(5)

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|        80|              8|              7|2014-03-01 00:00:02|
|        80|              8|              7|2014-03-01 00:01:03|
|        80|              8|              7|2014-03-01 00:02:03|
|        80|              8|              7|2014-03-01 00:03:02|
|        80|              8|              7|2014-03-01 00:04:03|
+----------+---------------+---------------+-------------------+
only showing top 5 rows



In [45]:
spark.sql('''
select max(day(TIME)) maxDayOfMonth 
from tb1'''
           ).show()

+-------------+
|maxDayOfMonth|
+-------------+
|           31|
+-------------+



Datetime format specifications:

http://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html 

In [51]:
spark.sql("select max(dayOfWeek(time)) as MAX_DOW from tb1;").show()

+-------+
|MAX_DOW|
+-------+
|      7|
+-------+



In [52]:
# spark.sql('''
# select max(from_unixtime(unix_timestamp(TIME,"yyyy-MM-dd"),"u")) maxDayOfWeek
# from tb1
# '''
#            ).show()

In [53]:
spark.sql('select distinct dayOfWeek(time) DoW from tb1').show()

+---+
|DoW|
+---+
|  1|
|  6|
|  3|
|  5|
|  4|
|  7|
|  2|
+---+



In [54]:
spark.sql('select collect_set(dayOfWeek(time)) DoW_SET from tb1').show(truncate=False)

+---------------------+
|DoW_SET              |
+---------------------+
|[1, 5, 2, 6, 3, 7, 4]|
+---------------------+



### Aggregation and sampling

` MIN , MAX , AVG , GROUPING SETS, ROLLUP , CUBE `

Sample data 1:

In [29]:
spark.sql('SELECT * FROM tb1 DISTRIBUTE BY RAND() SORT BY RAND() LIMIT 3').show()

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|        55|              4|             19|2014-04-28 08:50:03|
|        42|             11|              4|2014-06-19 04:44:02|
|        36|              7|              8|2014-03-15 11:23:02|
+----------+---------------+---------------+-------------------+



Sample data 2:

In [55]:
df3 = spark.sql('SELECT * FROM tb1 TABLESAMPLE(.01 PERCENT)')
df3.count()

1851

In [53]:
df3.show(5)

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|         2|             14|             12|2014-03-25 00:27:02|
|         2|             11|             16|2014-04-02 02:55:02|
|         2|              8|             19|2014-04-10 02:24:03|
|         2|              9|             18|2014-04-18 06:52:03|
|         2|             12|             15|2014-04-20 12:36:02|
+----------+---------------+---------------+-------------------+
only showing top 5 rows



In [56]:
spark.sql("show tables").show()

+--------+-------------+-----------+
|database|    tableName|isTemporary|
+--------+-------------+-----------+
| default|     employee|      false|
| default|  employee_07|      false|
| default|      licence|      false|
| default|logs_category|      false|
| default|     students|      false|
| default|  ward_counts|      false|
|        |          tb1|       true|
+--------+-------------+-----------+



In [57]:
spark.sql('''SELECT * 
               FROM tb1 
               SORT BY RAND() 
               LIMIT 3''').show()

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|         2|             11|             16|2014-03-10 02:14:02|
|         5|              8|             11|2014-05-14 21:04:02|
|         2|             17|             10|2014-05-15 01:13:02|
+----------+---------------+---------------+-------------------+



In [31]:
spark.sql('''
               SELECT docks_available - bikes_available AS free_slots
               FROM tb1
               LIMIT 3
            ''').show()

+----------+
|free_slots|
+----------+
|       3.0|
|       3.0|
|       3.0|
+----------+



- ` SORT BY ` in most cases preferrable to ` ORDER BY ` because it's faster (due to use of many reducers in ` SORT BY `)    


- Always avoid using ` ORDER BY ` in queries.

In [60]:
spark.sql('''SELECT dayofweek(time) DoW,
               round(AVG(docks_available - bikes_available),2) average_free_slots
               FROM tb1
               GROUP BY dayofweek(time)
               ORDER BY dayofweek(time) ASC
            ''').show()

+---+------------------+
|DoW|average_free_slots|
+---+------------------+
|  1|              0.67|
|  2|              0.84|
|  3|              0.92|
|  4|              0.86|
|  5|              0.82|
|  6|              0.77|
|  7|              0.65|
+---+------------------+



In [64]:
spark.sql('''SELECT date_format(time, 'E') DoW,
             round(AVG(docks_available - bikes_available),2) avearge_free_slots
             FROM tb1
             GROUP BY date_format(time, 'E')
            ''').show()

+---+------------------+
|DoW|avearge_free_slots|
+---+------------------+
|Sun|              0.67|
|Mon|              0.84|
|Thu|              0.82|
|Sat|              0.65|
|Wed|              0.86|
|Tue|              0.92|
|Fri|              0.77|
+---+------------------+



In [65]:
spark.sql('''SELECT date_format(time,"E") as day,
             count(*) as `count`
             FROM tb1
             GROUP BY date_format(time,"E")''').show()

+---+-------+
|day|  count|
+---+-------+
|Sun|2634647|
|Mon|2611193|
|Thu|2612085|
|Sat|2662475|
|Wed|2611430|
|Tue|2610988|
|Fri|2599392|
+---+-------+



Selecting columns that are not part of `GROUP BY` statement with:

> `colect_set(colName)`

In [71]:
spark.sql('''SELECT dayofweek(time) dayID,
             collect_set(date_format(time, 'E')) DoW,
             round(AVG(docks_available - bikes_available), 2) average_free_slots
             FROM tb1
             GROUP BY dayofweek(time)
             ORDER BY dayofweek(time) ASC
             ''').show()

+-----+-----+------------------+
|dayID|  DoW|average_free_slots|
+-----+-----+------------------+
|    1|[Sun]|              0.67|
|    2|[Mon]|              0.84|
|    3|[Tue]|              0.92|
|    4|[Wed]|              0.86|
|    5|[Thu]|              0.82|
|    6|[Fri]|              0.77|
|    7|[Sat]|              0.65|
+-----+-----+------------------+



In [78]:
spark.sql('''
with tb2 as
(select * from tb1 where dayofweek(time) = 1)
select collect_set(date_format(time, 'E')) DoW,
       AVG(docks_available - bikes_available) avearge_free_slots 
from tb2
''').show()

+-----+------------------+
|  DoW|avearge_free_slots|
+-----+------------------+
|[Sun]|0.6719503599533448|
+-----+------------------+



# Analytic functions

Used together with ` OVER , PARTITION BY , ORDER BY `  

Though analytic functions give aggregate results, they
do not group the result set. They return the group value multiple times with each
record.  


` Function (arg1,..., argn) OVER ([PARTITION BY <...>] [ORDER BY <....>]
[<window_clause>]) `

In [39]:
context.sql("select last_value(time) from tb1").show()

+-------------------+
|         last(time)|
+-------------------+
|2014-08-31 23:59:03|
+-------------------+



In [48]:
context.sql("select max(time) from tb1").show()

+-------------------+
|          max(time)|
+-------------------+
|2014-08-31 23:59:03|
+-------------------+



Convert table to DF

In [37]:
df2 = context.table('tb1')
type(df2)

pyspark.sql.dataframe.DataFrame

In [38]:
df2.show(5)

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|         2|             12|             15|2014-03-01 00:00:02|
|         2|             12|             15|2014-03-01 00:01:03|
|         2|             12|             15|2014-03-01 00:02:03|
|         2|             12|             15|2014-03-01 00:03:02|
|         2|             12|             15|2014-03-01 00:04:03|
+----------+---------------+---------------+-------------------+
only showing top 5 rows

